# Sistema de Recomendación

## 1. Extraemos el DF limpio de Fases previas. 

In [1]:
# imports
import pandas as pd
from tabulate import tabulate

from funciones_files import ask_tags_comma_separated
from funciones_scraping import get_languages

df = pd.read_csv('../output/df-limpio-iter3.csv', index_col=0)
df.shape

(11428, 57)

In [2]:
languages = get_languages()
df['linkedin'] = df['linkedin'].fillna('')
df['github'] = df['github'].fillna('')

In [3]:
def normalize_columns(data, columns): 
    for col in columns: 
        if data[col].max() > 0: 
            data[col] = data[col]/data[col].max()
    return data

df_norm = normalize_columns(df, languages)
df_norm.reset_index(drop=True, inplace=True)


In [4]:
df_norm.describe()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,days of activity
count,11428.000000,11428.0,11428.000000,11428.0,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,...,11428.0,11428.000000,11428.0,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000
mean,93.289902,0.0,0.001295,0.0,0.0,0.001161,0.000957,0.143857,13.353255,0.000312,...,0.0,0.000551,0.0,0.009283,0.000642,102.076041,2.412496,1.488799,0.001119,496.643420
std,232.583439,0.0,0.019172,0.0,0.0,0.017324,0.014934,0.843151,173.395939,0.011377,...,0.0,0.012697,0.0,0.056273,0.015932,289.016703,2.597626,39.647146,0.020352,461.830188
min,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,91.000000
50%,15.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,28.000000,2.000000,0.000000,0.000000,395.000000
75%,72.250000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,76.000000,3.000000,0.000000,0.000000,791.000000
max,10610.000000,0.0,1.000000,0.0,0.0,1.000000,1.000000,29.000000,8962.000000,1.000000,...,0.0,1.000000,0.0,1.000000,1.000000,7811.000000,38.000000,2927.000000,1.000000,2252.000000


## Toma de requisitos

Se pregunta al usuario por los tags que está buscando entre los candidatos

In [5]:
'''
def ask_languages(languages=languages): 
    while True: 
        try: 
            res = input('Introduce los elementos separados por coma:').lower().split(',')
            res = [r.strip() for r in res]
            break
        except: 
            print('Fallo en la captura de requisitos. Inténtalo de nuevo')
    return res
    
    '''

"\ndef ask_languages(languages=languages): \n    while True: \n        try: \n            res = input('Introduce los elementos separados por coma:').lower().split(',')\n            res = [r.strip() for r in res]\n            break\n        except: \n            print('Fallo en la captura de requisitos. Inténtalo de nuevo')\n    return res\n    \n    "

In [6]:
# ask_languages()
skills = ask_tags_comma_separated('Introduce los lenguajes buscados separados por coma:')

Introduce los lenguajes buscados separados por coma:python, java


In [7]:
def get_ask_vector(skills, languages): 
    res = sorted([(l, 1) if l in skills else (l, 0) for l in languages])
    return [v for k, v in res]
# get_ask_vector(['python', 'c'], get_languages())       

def distacia_euclidea(v1, v2): 
    return sum([(x1-x2)**2 for x1, x2 in zip(v1, v2)])**0.5
# distacia_euclidea([3, 4], [0, 0])

def get_languages_vector(row, languages):
    # return [(k, v) for k, v in row.items() if k in languages]
    res = sorted([(k, v) for k, v in row.items() if k in languages])
    return [v for k, v in res]
# get_languages_vector(df_norm.loc[4, :], get_languages())

v1 = get_ask_vector(skills, languages)

## Cálculo de distancias

In [8]:
distancias = []
for i in df_norm.index.values: 
    distancias.append( distacia_euclidea(v1, get_languages_vector(df_norm.loc[i, :], languages)) )
len(distancias)    

11428

In [9]:
df_norm['distancia'] = distancias

In [10]:
df_sort = df_norm.sort_values(by=['distancia'])
df_sort.head()

,allies,bf (beta),c,c#,c++,clojure,coffeescript,collections,comments,crystal,...,swift,total completed kata,total languages trained,translations,typescript,username,github,linkedin,days of activity,distancia
3330,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,7358.0,0.0,...,0.0,3297.0,9.0,190.0,0.0,Blind4Basics,https://github.com/Blind4Basics,,821,0.200218
5590,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,170.0,5.0,7.0,0.0,BabyPowder,,,184,1.044817
3150,616.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,295.0,0.0,...,0.0,631.0,4.0,3.0,0.0,ecolban,,,1310,1.051513
4167,507.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,55.0,6.0,0.0,0.0,Gperezg,https://github.com/GerryGPG,,184,1.068237
10861,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1369.0,0.0,...,0.0,2576.0,1.0,1.0,0.0,lechevalier,,,1157,1.069890


In [11]:
users_sorted = df_sort[['username', 'github', 'linkedin']]

len(users_sorted)

11428

## Persistencia de la recomendación

In [12]:
path = '../output/recomendation.txt'
with open('../output/recomendation.txt', 'w') as f: 
    f.write(tabulate(users_sorted, headers=['username', 'github', 'linkedin']))
# save_set_users(users_sorted, filename=path)

In [13]:
users_sorted.head()

,username,github,linkedin
3330,Blind4Basics,https://github.com/Blind4Basics,
5590,BabyPowder,,
3150,ecolban,,
4167,Gperezg,https://github.com/GerryGPG,
10861,lechevalier,,
